In [1]:
import json
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import copy
from utils.utils import *
import pandas as pd

In [2]:
method_dict = {"MC": "Random Sample","random sample": "Random Sample", "surr ensemble": "ASE Class Score 0", "ASE": "ASE Class Score 0", "ASE_regression": "ASE Regression Score 0", 
               "ASE_all": "ASE All Score 0", "ASE_class_score_1": "ASE Class Score -1", "ASE_reg_score_1": "ASE Regression Score -1", "ASE_all_score_1": "ASE All Score -1", "MPL": "MPL"}
metric_dict = {'loss': 'Loss', 'mAP': 'mAP', "mAR_1000": "mAR"}
def add_entries(file):
    #columns=['active_test_type','sample_size','loss','mAP',"AP_50", "AP_75", "mAP_samll", "mAP_medium", "mAP_large", "mAR_1", "mAR_10", "mAR_1000", "mAR_small", "mAR_medium", "mAR_large"]
    with open(file, 'r') as openfile:
        json_object = json.load(openfile)
    df = None
    for key,value in json_object.items():
        if 'test_stats' in value.keys():
            del value['test_stats']
        # value['active_test_type'] = method_dict[value['active_test_type']]
        # if value['sample_size'] < 180:
        #     continue
        temp_df = pd.DataFrame(value, index=[key])
        if df is None:
            df = temp_df
        else:
            df = df.append(temp_df,ignore_index=True)
    return df

In [3]:
def plot_figure(data, baseline_data, y_metric, x_label_name = "# of Acquired Images"):
    f = plt.figure(figsize=(10, 8))
    font_size = 25
    with sns.axes_style("darkgrid"):
        sns.lineplot(data=data, \
                     x='sample_size', y=y_metric, \
                     hue='active_test_type', errorbar=('ci', 100), lw=3)
    plt.plot([data['sample_size'][0], data['sample_size'][len(data['sample_size'])-1]], [baseline_data[y_metric], baseline_data[y_metric]], lw=3, label="Whole Data Set")
    plt.title("Active Testing", fontsize=30)
    plt.xlabel(x_label_name, fontsize=font_size)# x轴标签
    plt.ylabel(metric_dict[y_metric], fontsize=font_size)# y轴标签
    # pic_name = scenario+'.png'
    plt.legend(fontsize = 18)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.tight_layout()
    #plt.ylim(-1600,-800)
    #plt.savefig(pic_name,dpi=800)
    plt.show()
    
# def plot_all_diff_precentage_figure(input_data, baseline_data, sample_nums, y_metric, x_label_name = "% of Acquired Images"):
#     data = copy.deepcopy(input_data)
#     f = plt.figure(figsize=(10, 8))
#     font_size = 25
#     data[y_metric] -= baseline_data[y_metric]
#     data[y_metric] = data[y_metric] / baseline_data[y_metric] * 100
#     data['sample_size'] = data['sample_size'] / sample_nums * 100
#     with sns.axes_style("darkgrid"):
#         sns.lineplot(data=data, \
#                      x='sample_size', y=y_metric, \
#                      hue='active_test_type', errorbar=('ci', 100), lw=3)
        
#     x_line = [data['sample_size'][0], data['sample_size'][data['sample_size'].shape[0]-1]]
#     y_line = [0, 0]
#     plt.plot(x_line, y_line, 'r')
#     plt.title("Active Testing", fontsize=30)
#     plt.xlabel(x_label_name, fontsize=font_size)# x轴标签
#     plt.ylabel("% Difference Rate to Full Test " + metric_dict[y_metric], fontsize=font_size)# y轴标签
#     # pic_name = scenario+'.png'
#     plt.legend(fontsize = 18)
#     plt.xticks(fontsize=20)
#     plt.yticks(fontsize=20)
#     plt.tight_layout()
#     #plt.ylim(-1600,-800)
#     #plt.savefig(pic_name,dpi=800)
#     plt.show()

def plot_all_diff_precentage_figure(input_data, baseline_data, sample_nums, y_metric, x_label_name = "% of Acquired Images"):
    data = copy.deepcopy(input_data)
    f = plt.figure(figsize=(10, 8))
    font_size = 25
    data[y_metric] = abs(data[y_metric] - baseline_data[y_metric])
    data[y_metric] = data[y_metric] / baseline_data[y_metric] * 100
    data['sample_size'] = data['sample_size'] / sample_nums * 100
    with sns.axes_style("darkgrid"):
        sns.lineplot(data=data, \
                     x='sample_size', y=y_metric, \
                     hue='active_test_type', errorbar=('sd', 1), lw=3)
        
    # x_line = [data['sample_size'][0], data['sample_size'][data['sample_size'].shape[0]-1]]
    # y_line = [0, 0]
    # plt.plot(x_line, y_line, 'r')
    plt.title("Active Testing", fontsize=30)
    plt.xlabel(x_label_name, fontsize=font_size)# x轴标签
    plt.ylabel("% ER ", fontsize=font_size)# y轴标签
    # pic_name = scenario+'.png'
    plt.legend(fontsize = 18)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.tight_layout()
    # plt.style.use("fivethirtyeight")
    ax = plt.gca()
    ax.set_facecolor('none')
    #plt.ylim(-1600,-800)
    #plt.savefig(pic_name,dpi=800)
    plt.show()

def plot_diff_figure(input_data, baseline_data, y_metric, x_label_name = "# of Acquired Images"):
    data = add_entries(input_data)
    f = plt.figure(figsize=(10, 8))
    font_size = 25
    data[y_metric] -= baseline_data[y_metric]
    with sns.axes_style("darkgrid"):
        sns.lineplot(data=data, \
                     x='sample_size', y=y_metric, \
                     hue='active_test_type', errorbar=('ci', 100), lw=3)
    plt.title("Active Testing", fontsize=30)
    plt.xlabel(x_label_name, fontsize=font_size)# x轴标签
    plt.ylabel(metric_dict[y_metric] + " diff", fontsize=font_size)# y轴标签
    # pic_name = scenario+'.png'
    plt.legend(fontsize = 18)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.tight_layout()
    #plt.ylim(-1600,-800)
    #plt.savefig(pic_name,dpi=800)
    plt.show()
    
def plot_diff_precentage_figure(data_path, baseline_data, y_metric, x_label_name = "# of Acquired Images"):
    data = add_entries(data_path)
    f = plt.figure(figsize=(10, 8))
    font_size = 25
    data[y_metric] -= baseline_data[y_metric]
    data[y_metric] = data[y_metric] / baseline_data[y_metric]
    with sns.axes_style("darkgrid"):
        sns.lineplot(data=data, \
                     x='sample_size', y=y_metric, \
                     hue='active_test_type', errorbar=('ci', 100), lw=3)
    plt.title("Active Testing", fontsize=30)
    plt.xlabel(x_label_name, fontsize=font_size)# x轴标签
    plt.ylabel(metric_dict[y_metric] + " Diff %", fontsize=font_size)# y轴标签
    plt.legend(fontsize = 18)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.tight_layout()
    plt.show()
    
def plot_diff_precentage_figure_comp(input_data, baseline_data, y_metric, x_label_name = "# of Acquired Images"):
    data = copy.deepcopy(input_data)
    f = plt.figure(figsize=(10, 8))
    font_size = 25
    data[y_metric] -= baseline_data[y_metric]
    data[y_metric] = data[y_metric] / baseline_data[y_metric]
    with sns.axes_style("darkgrid"):
        sns.lineplot(data=data, \
                     x='sample_size', y=y_metric, \
                     hue='active_test_type', errorbar=('ci', 100), lw=3)
    plt.title("Active Testing", fontsize=30)
    plt.xlabel(x_label_name, fontsize=font_size)# x轴标签
    plt.ylabel(metric_dict[y_metric] + " Diff %", fontsize=font_size)# y轴标签
    plt.legend(fontsize = 18)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.tight_layout()
    plt.show()

def read_baseline_data(data_path):
    with open(data_path, 'r') as openfile:
        none_swin_data = json.load(openfile)
    return none_swin_data["0"]

def print_mean_std(path, test_whole_data):
    data = add_entries(path)
    min_size = int(min_size_p * sample_size_threshold)
    max_size = int(max_size_p * sample_size_threshold)
    data = data[data['sample_size'] >= min_size]
    data = data[data['sample_size'] <= max_size]
    data = data['loss'].values
    data = data.reshape(3, data.shape[0]//3)
    data = (np.abs(data - test_whole_data) / test_whole_data) * 100
    mean = data.mean()
    std = data.mean(axis=1).std()
    print(f"{path}: mean {mean:.2f}, std {std:.2f}")

## Image based

In [4]:
model_dataset = "UNet_VOC"

min_size_p = 0.01
max_size_p = 0.05

base_path = f"./pro_data/{model_dataset}/val/"
sample_size_threshold = np_read(base_path + "image_true_losses.npy").shape[0]
result_json_path = f"./results/{model_dataset}/image_based_active_testing/"
test_whole_data = read_baseline_data(result_json_path + "None.json")['loss']
print_mean_std(result_json_path + "random_sample_3_runs.json", test_whole_data)
print_mean_std(result_json_path + "ViT_range_all_runs_10000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_range_image_runs_10000.json", test_whole_data)
print_mean_std(result_json_path + "ResNet_range_image_runs.json", test_whole_data)
print_mean_std(result_json_path + "MLP_range_image_runs.json", test_whole_data)
print_mean_std(result_json_path + "ViT_regress_range_image_runs_20000.json", test_whole_data)

./results/UNet_VOC/image_based_active_testing/random_sample_3_runs.json: mean 11.60, std 3.74
./results/UNet_VOC/image_based_active_testing/ViT_range_all_runs_10000.json: mean 6.21, std 0.79
./results/UNet_VOC/image_based_active_testing/ViT_range_image_runs_10000.json: mean 11.95, std 4.20
./results/UNet_VOC/image_based_active_testing/ResNet_range_image_runs.json: mean 5.69, std 1.06
./results/UNet_VOC/image_based_active_testing/MLP_range_image_runs.json: mean 10.73, std 7.03
./results/UNet_VOC/image_based_active_testing/ViT_regress_range_image_runs_20000.json: mean 8.02, std 0.68


## Region based

In [9]:
min_size_p = 0.0001
max_size_p = 0.0008
base_path = f"./pro_data/{model_dataset}/val/"
sample_size_threshold = np_read(base_path + "region_16_16_true_losses.npy").shape[0]
result_json_path = f"./results/{model_dataset}/region_16_16_active_testing/"
test_whole_data = read_baseline_data(result_json_path + "None.json")['loss']
print_mean_std(result_json_path + "random_sample_3_runs.json", test_whole_data)
print_mean_std(result_json_path + "ViT_range_region_runs_10000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_range_all_runs_10000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_without_entropy_runs_10000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_without_image_runs_10000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_without_output_runs_10000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_UNet_VOC_class_30_runs_20000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_UNet_VOC_class_40_runs_20000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_UNet_VOC_class_60_runs_20000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_UNet_VOC_class_70_runs_20000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_UNet_VOC_class_80_runs_20000.json", test_whole_data)
print_mean_std(result_json_path + "ViT_UNet_VOC_class_90_runs_20000.json", test_whole_data)

./results/UNet_VOC/region_16_16_active_testing/random_sample_3_runs.json: mean 4.96, std 0.56
./results/UNet_VOC/region_16_16_active_testing/ViT_range_region_runs_10000.json: mean 8.97, std 3.87
./results/UNet_VOC/region_16_16_active_testing/ViT_range_all_runs_10000.json: mean 7.43, std 3.12
./results/UNet_VOC/region_16_16_active_testing/ViT_without_entropy_runs_10000.json: mean 14.36, std 5.98
./results/UNet_VOC/region_16_16_active_testing/ViT_without_image_runs_10000.json: mean 8.54, std 3.19
./results/UNet_VOC/region_16_16_active_testing/ViT_without_output_runs_10000.json: mean 9.38, std 3.26
./results/UNet_VOC/region_16_16_active_testing/ViT_UNet_VOC_class_30_runs_20000.json: mean 10.11, std 5.10
./results/UNet_VOC/region_16_16_active_testing/ViT_UNet_VOC_class_40_runs_20000.json: mean 4.27, std 1.40
./results/UNet_VOC/region_16_16_active_testing/ViT_UNet_VOC_class_60_runs_20000.json: mean 8.88, std 2.96
./results/UNet_VOC/region_16_16_active_testing/ViT_UNet_VOC_class_70_runs_2000

In [ ]:
for classes in [30,40,60,70,80,90]:
    for step in [5000, 10000, 15000, 20000]:
        print_mean_std(result_json_path + f"ViT_UNet_VOC_class_{classes}_runs_{step}.json", test_whole_data)